# Generate exchange data using dedicated service - F#

### Overview
Reference data encompasses a wide range of specification data about:
* financial instrument such as asset class, symbol, maturity, etc.
* counterparties such as issuer information.
* pricing such as Open, high low and close prices.

The challenge with reference data is that it tends to be sourced from multiple sources: internal, counterparties and providers.

Reference data dedicated service is a result of various data sources and fields sourcing, validation, cross-checking and normalization.<br>

This sample requests **on-demand** exchange data and enables fields mapping by calling a dedicated static data service, making available clean data.

### Inputs/outputs
Exchange data sample is designed to help you searching and requesting exchange reference data by using a wide range of commonly used fields.

It takes exchange identifier as per input filter and returns the *entries* matching the *request*:
* List of instruments' identifiers
* Countries
* Currencies 
* Tick size rules
* etc. 

### Services used
This sample uses *gRPC requests* in order to retrieve exchange reference data from the hosted service. The queried endpoint in this script are:
* *StaticDataService*: to directly retrieve reference data objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.StaticData.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*


# Run Exchange data sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 2.33.*-pre*"
#r "nuget: CsvHelper"

In [ ]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.StaticData.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
// Import/export csv files
open CsvHelper
open System.IO
open System.Globalization

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Request creation

The following code snippet enables to select the **exchange** by its *Market Identifier Code*:

In [ ]:
// Set exchange
let exchange = "XNGS"

The following code snippets call the service, generate the request and return the reply: 

In [ ]:
// Instantiate the service
let service = new StaticDataService.StaticDataServiceClient(channel)

// Generate request
let request = new StaticDataRequest ( AssetType = AssetType.Equity, Exchange = exchange, Count = 100 )

// Call the service
let reply = service.StaticData(request, headers);

// Display the results
//display(reply.Equities)

### Step 4: Retrieve data

#### 4.1 Retrieve instruments' identifiers (mapping)

In [ ]:
type Instrument = 
    {
        Ticker: string
        Name : string
        Isin : string
        Cusip : string
        Sedol : string
        Bloomberg : string
    }

let instruments = reply.Equities |> Seq.map (fun e -> { Ticker = e.Identifier.Ticker; Name = e.Name; Isin = e.Isin; Cusip = e.Cusip; Sedol = e.Sedol; Bloomberg = if e.Mapping.ContainsKey("Bloomberg") then e.Mapping.["Bloomberg"] else String.Empty })

In [ ]:
display(instruments)

The following code snippet exports data to *csv* file:

In [ ]:
let d = new DirectoryInfo("output") 
d.Create()
let writer = new StreamWriter($"output/{exchange}.csv")
let csv = new CsvWriter(writer, CultureInfo.InvariantCulture)
csv.WriteRecords(instruments)


#### 4.2 Retrieve countries

In [ ]:
reply.Equities |> Seq.map (fun e -> e.Country) |> Seq.distinct

#### 4.3 Retrieve currencies

In [ ]:
reply.Equities |> Seq.map (fun e -> e.Currency) |> Seq.distinct

#### 4.4 Retrieve tick size rules

In [ ]:
reply.Equities |> Seq.map (fun e -> e.TickSizeRule) |> Seq.distinct